In [1]:
!pip install panda
import pandas as pd
!pip install pyxlsb

  Preparing metadata (setup.py) ... done
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7239 sha256=5f9d0bc16f28ab66b1bfe928547f1860cd34408598465762e85a5b6c6426d16b
  Stored in directory: /root/.cache/pip/wheels/df/5c/39/36f8dae25a1e88d6ec4411dec4a143781e64fdff6897758eec
Successfully built panda


In [17]:
file_path = "/content/D2h_sample_45K_data_for_Churn_prediction_modelling (1).xlsb"

In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

# Load dataset
data_file = '/content/D2h_sample_45K_data_for_Churn_prediction_modelling.xlsb'
df = pd.read_excel(data_file, engine='pyxlsb')


In [70]:
# Data Analysis
def data_analysis(df):
    print("Dataset Overview:")
    print(df.info())
    print("\nSummary Statistics:")
    print(df.describe())
    print("\nMissing Values:")
    print(df.isnull().sum())

data_analysis(df)


Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45419 entries, 0 to 45418
Data columns (total 39 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   smsid                                      45419 non-null  int64  
 1   Pincode                                    45419 non-null  int64  
 2   Email_Check                                45419 non-null  int64  
 3   RMN_Flag                                   45419 non-null  int64  
 4   AON                                        45419 non-null  int64  
 5   LangaugeZone                               45419 non-null  object 
 6   LanguagePrintName                          45419 non-null  object 
 7   DASType                                    45419 non-null  int64  
 8   Model_type                                 45419 non-null  object 
 9   Boxtype                                    45419 non-null  object 
 10  Mode

In [75]:
import pandas as pd

# Load the dataset
file_path = "/content/D2h_sample_45K_data_for_Churn_prediction_modelling.xlsb"

# Inspect available sheets
xlsb_file = pd.ExcelFile(file_path, engine="pyxlsb")
xlsb_file.sheet_names


['Sheet1']

In [76]:
# Load the dataset from "Sheet1"
df = pd.read_excel(file_path, sheet_name="Sheet1", engine="pyxlsb")

# Display the first few rows of the dataset
df.head()


,smsid,Pincode,Email_Check,RMN_Flag,AON,LangaugeZone,LanguagePrintName,DASType,Model_type,Boxtype,...,Total_VAS_Count,VAS_Price,Total_Add_On_Count,Upgrade,DownGrade,OB_calls_in_last_1_month,relocation_flag,SwapCount,BSP_Plan_Activated,TotalChildCount
0,2780601,781335,1,1,6,NORTH,HINDI/PUNJ,5,SD,MPEG-4,...,0.0,NaN,NaN,0,0,1,NaN,NaN,0,NaN
1,2782701,641109,1,1,6,TAMIL,TAMIL,2,SD,MPEG-4,...,0.0,NaN,3.0,0,0,1,NaN,NaN,0,NaN
2,2782801,401101,1,1,6,NORTH,MARATHI,2,HD,MPEG-4,...,1.0,NaN,4.0,0,0,1,NaN,NaN,0,NaN
3,2785001,752108,1,1,6,NORTH,HINDI/PUNJ,5,HD,MPEG-4,...,0.0,NaN,20.0,1,1,1,NaN,NaN,0,NaN
4,2785101,421201,1,1,6,NORTH,HINDI/PUNJ,2,HD,MPEG-4,...,0.0,NaN,10.0,0,0,1,NaN,NaN,0,NaN


In [77]:
# Check for missing values in the dataset
missing_values = df.isnull().sum()

# Display columns with missing values
missing_values[missing_values > 0]


,0
ticketsize,17567
CSM_COLOR,26
RADA_COLOR,12
Active_Days_mar24,2157
Active_Days_Jan24,4220
Active_Days_Feb24,3685
TotalComplaintCount,30315
Resolved_After_72Hrs,30315
Last_IB_Call_Disposition,30315
No_of_time_DA_in_last_6_months,14027


In [78]:
# Identify numeric and categorical columns
numeric_cols = df.select_dtypes(include=["number"]).columns
categorical_cols = df.select_dtypes(exclude=["number"]).columns

# Fill numeric columns with median values
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Fill categorical columns with mode values
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Verify if any missing values remain
df.isnull().sum().sum()


0

In [79]:
# Define the churn prediction function based on user activity
df["Churn Prediction"] = df[["Active_Days_Jan24", "Active_Days_Feb24", "Active_Days_mar24"]].max(axis=1).apply(
    lambda x: "Active" if x > 20 else "Non-Active"
)

# Display value counts for the new column
df["Churn Prediction"].value_counts()


,count
Churn Prediction,
Active,45034
Non-Active,385


In [81]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Selecting features (excluding user identifiers and directly derived columns)
features = df.drop(columns=["smsid", "Churn Prediction"])

# Encode categorical variables
label_encoders = {}
for col in features.select_dtypes(exclude=["number"]).columns:
    # Convert the column to string type before encoding
    features[col] = features[col].astype(str)
    le = LabelEncoder()
    features[col] = le.fit_transform(features[col])
    label_encoders[col] = le  # Store encoder for future use

# Define target variable
target = df["Churn Prediction"].map({"Active": 1, "Non-Active": 0})  # Convert labels to binary

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Display dataset shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((36335, 38), (9084, 38), (36335,), (9084,))

In [82]:
# Identify columns with mixed types
mixed_type_cols = [col for col in features.columns if features[col].apply(lambda x: isinstance(x, str)).any() and features[col].apply(lambda x: isinstance(x, (int, float))).any()]

# Convert all mixed-type columns to string type for consistency
for col in mixed_type_cols:
    features[col] = features[col].astype(str)

# Retry encoding categorical variables
label_encoders = {}
for col in features.select_dtypes(exclude=["number"]).columns:
    le = LabelEncoder()
    features[col] = le.fit_transform(features[col])
    label_encoders[col] = le  # Store encoder for future use

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Display dataset shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((36335, 38), (9084, 38), (36335,), (9084,))

In [83]:
# Redefine the target variable after handling data type issues
target = df["Churn Prediction"].map({"Active": 1, "Non-Active": 0})  # Convert labels to binary

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Display dataset shapes
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((36335, 38), (9084, 38), (36335,), (9084,))

In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the mean absolute error
mae = mean_absolute_error(y_test, y_pred)
mae


0.00011008366358432408

In [85]:
# Get predicted probabilities for the positive class (Active = 1)
df.loc[X_test.index, "Predicted Score"] = model.predict_proba(X_test)[:, 1]

# Display sample predictions
df.loc[X_test.index, ["Churn Prediction", "Predicted Score"]].head()


,Churn Prediction,Predicted Score
3939,Active,1.00
24569,Active,0.99
40981,Active,1.00
36695,Active,1.00
41211,Active,1.00
